<a href="https://colab.research.google.com/github/gaeun1961/DS_Basic_Analysis_Python/blob/main/02_%EA%B3%B5%EA%B3%B5%EB%8D%B0%EC%9D%B4%ED%84%B0_api_%EA%B8%B0%EB%B0%98_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import urllib.request
import datetime
import time
import json
import pandas as pd

In [79]:
from google.colab import userdata
ServiceKey = userdata.get('data_kr')

In [74]:
# [CODE 1]
def getRequestUrl(url):
  req = urllib.request.Request(url)
  try:
    response = urllib.request.urlopen(req)
    if response.getcode() == 200:
      print("[%s] Url Request Success" % datetime.datetime.now())
      return response.read().decode('utf-8')
  except Exception as e:
    print(e)
    print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
    return None

In [75]:
#[CODE 2]
def getTourismStatsItem(yyyymm, national_code, ed_cd):
    service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"
    parameters = "?_type=json&serviceKey=" + ServiceKey   #인증키
    parameters += "&YM=" + yyyymm
    parameters += "&NAT_CD=" + national_code
    parameters += "&ED_CD=" + ed_cd
    url = service_url + parameters
    print(url) #액세스 거부 여부 확인용 출력
    responseDecode = getRequestUrl(url)   #[CODE 1]

    if (responseDecode == None):
        return None
    else:
         return json.loads(responseDecode)

In [76]:
#[CODE 3]
def getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear):
    jsonResult = []
    result = []
    natName=''
    dataEND = "{0}{1:0>2}".format(str(nEndYear), str(12)) #데이터 끝 초기화
    isDataEnd = 0 #데이터 끝 확인용 flag 초기화

    for year in range(nStartYear, nEndYear+1):
        for month in range(1, 13):
            if(isDataEnd == 1): break    #수집기간이 남았어도 데이터 끝이면 작업 중지.

            yyyymm = "{0}{1:0>2}".format(str(year), str(month))
            jsonData = getTourismStatsItem(yyyymm, nat_cd, ed_cd) #[CODE 2]

            if (jsonData['response']['header']['resultMsg'] == 'OK'):
                # 아직 입력된 범위까지 수집하지 않았지만, 더이상 제공되는 데이터가 없는 경우 -------------------
                if jsonData['response']['body']['items'] == '':
                    isDataEnd = 1 #데이터 끝 flag 설정
                    if (month-1)==0:  #마지막 데이터가 12월인데, 현재 month가 다음해 1월인 경우
                        year = year-1
                        month = 13

                    dataEND = "{0}{1:0>2}".format(str(year), str(month-1))
                    print("데이터 없음.... \n제공되는 통계 데이터는 %s년 %s월까지입니다.\n"
                              %(str(year), str(month-1)), '-'*70)
                    jsonData = getTourismStatsItem(dataEND, nat_cd, ed_cd) #출력용으로 마지막 데이터 다시 호출
                    break

                #반환할 데이터 항목을 추출하여 저장
                natName = jsonData['response']['body']['items']['item']['natKorNm']
                natName = natName.replace(' ', '')
                num = jsonData['response']['body']['items']['item']['num']
                ed = jsonData['response']['body']['items']['item']['ed']
                print('[ %s_%s : %s ]' %(natName, yyyymm, num))
                print('-'*70)
                jsonResult.append({'nat_name': natName, 'nat_cd': nat_cd,
                                 'yyyymm': yyyymm, 'visit_cnt': num})
                result.append([natName, nat_cd, yyyymm, num])

    #마지막 jsonData를 확인용으로 출력하기......................................................
    print ('<마지막 jsonData>\n', json.dumps(jsonData, indent=4, sort_keys=True, ensure_ascii=False))

    return (jsonResult, result, natName, ed, dataEND)


In [78]:
# [CODE 0]
def main():
  jsonResult = []
  result = []
  natName = ''

  print("<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>")
  nat_cd=input('국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) : ')
  nStartYear = int(input('데이터를 몇 년부터 수집할까요? : '))
  nEndYear = int(input('데이터를 몇 년까지 수집할까요? : '))
  ed_cd = "E"

  try:
    jsonResult, result, natName, ed, dataEND = getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear)
    if (natName == ''):
      print('데이터가 전달되지 않았습니다. 공공데이터포털의 서비스 상태를 확인하기 바랍니다.')
    else:

      # 파일 저장 1: json 파일
      with open('./%s_%s_%d_%s.json' % (natName, ed, nStartYear, dataEND), 'w', encoding='utf8') as outfile:
       jsonFile = json.dumps(jsonResult,indent = 4, sort_keys = True, ensure_ascii = False)
       outfile.write(jsonFile)

      # 파일 저장2: csv 파일
      columns = ["입국자국가", "국가코드", "입국연월", "입국자수"]
      result_df = pd.DataFrame(result, columns = columns)
      result_df.to_csv('./%s_%s_%d_%s.csv' % (natName, ed, nStartYear, dataEND), index=False, encoding='cp949')
  except TypeError as e:
    print(f"데이터 수집 중 오류가 발생했습니다: {e}")
    print("API 호출에 문제가 있거나 응답 형식이 예상과 다릅니다. 네트워크 연결 및 API 상태를 확인해 주세요.")

if __name__ == '__main__':
  main()

<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>
국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) : 130
데이터를 몇 년부터 수집할까요? : 2020
데이터를 몇 년까지 수집할까요? : 2022
http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList?_type=json&serviceKey=RKp7mtfryeAiTFH74uWHBriQrfiZrYTEp1ujt5BSi9y7eqmJvOMV1dYp3KfFSp4uMf19WPjBjwz5vC%2Fe0UbDfg%3D%3D&YM=202001&NAT_CD=130&ED_CD=E
<urlopen error [Errno 110] Connection timed out>
[2025-11-28 14:13:10.845735] Error for URL : http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList?_type=json&serviceKey=RKp7mtfryeAiTFH74uWHBriQrfiZrYTEp1ujt5BSi9y7eqmJvOMV1dYp3KfFSp4uMf19WPjBjwz5vC%2Fe0UbDfg%3D%3D&YM=202001&NAT_CD=130&ED_CD=E
데이터 수집 중 오류가 발생했습니다: 'NoneType' object is not subscriptable
API 호출에 문제가 있거나 응답 형식이 예상과 다릅니다. 네트워크 연결 및 API 상태를 확인해 주세요.
